# Анализ БД крупного сервиса для чтения книг по подписке

**Цель:** проанализировать базу данных, которая помогут сформулировать ценностное предложение для нового продукта.

**Задания:**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


___
## Чтение файлов и предобработка данных
На данном этапе будут выгружены файлы, проведён первичный анализ и сделана предобработка данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import warnings

# избавимся от вывода ошибок, так код будет выглядеть красивее
warnings.simplefilter("ignore")

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# создадим функцию для чтения файлов
def reader(data):
    query = '''SELECT * FROM {}'''.format(data)
    return pd.io.sql.read_sql(query, con = engine)

In [4]:
# запишем данные с переменные
books = reader('books')
authors = reader('authors')
ratings = reader('ratings')
reviews = reader('reviews')
publishers = reader('publishers')

In [5]:
# создадим функцию для применения query
def query(func):
    return pd.io.sql.read_sql(func, con = engine)

In [6]:
#функция для предобработки данных

def preprocessing(file, file_name):
    print('Посмотрим первые строки датафрейма', file_name)
    display(file.head())
    print('___________________________')
    print('')
    
    print('Информация о таблице', file_name)
    print(file.info())
    print('___________________________')
    print('')
    
    print('Распределение данных в таблице', file_name)
    print(file.describe())
    print('___________________________')
    print('')  
    
    print('Наличие явных дубликатов в', file_name)
    print(file.duplicated().sum())
    print('___________________________')
    print('')
    
    print('Наличие пропусков в таблице', file_name)
    print(file.isna().sum())

In [7]:
preprocessing(books, 'books')

Посмотрим первые строки датафрейма books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


___________________________

Информация о таблице books
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None
___________________________

Распределение данных в таблице books
           book_id    author_id   num_pages  publisher_id
count  1000.000000  1000.000000  1000.00000   1000.000000
mean    500.500000   320.417000   389.11100    171.270000
std     288.819436   181.620172   229.39014     99.082685
min       1.000000     1.000000    14.00000      1.000000
25%     250.750000   162.750000   249.00000     83.000000
5

**Данные выгружены корретно, пропусков и дубликатов нет, однако тип данных не совсем соответствует. Приведём дату к нужному формату.**

In [8]:
books['publication_date'] = pd.to_datetime(books['publication_date'])

In [9]:
preprocessing(authors, 'authors')

Посмотрим первые строки датафрейма authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


___________________________

Информация о таблице authors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None
___________________________

Распределение данных в таблице authors
        author_id
count  636.000000
mean   318.500000
std    183.741666
min      1.000000
25%    159.750000
50%    318.500000
75%    477.250000
max    636.000000
___________________________

Наличие явных дубликатов в authors
0
___________________________

Наличие пропусков в таблице authors
author_id    0
author       0
dtype: int64


In [10]:
preprocessing(ratings, 'ratings')

Посмотрим первые строки датафрейма ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


___________________________

Информация о таблице ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None
___________________________

Распределение данных в таблице ratings
         rating_id      book_id       rating
count  6456.000000  6456.000000  6456.000000
mean   3228.500000   510.574195     3.928284
std    1863.831001   284.141636     0.943303
min       1.000000     1.000000     1.000000
25%    1614.750000   291.000000     3.000000
50%    3228.500000   506.000000     4.000000
75%    4842.250000   750.000000     5.000000
max    6456.000000  1000.000000     5.000000
___________________________

Наличие явных дубликатов в ratings
0
______

In [11]:
preprocessing(reviews, 'reviews')

Посмотрим первые строки датафрейма reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


___________________________

Информация о таблице reviews
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None
___________________________

Распределение данных в таблице reviews
         review_id      book_id
count  2793.000000  2793.000000
mean   1397.000000   504.693161
std     806.413976   288.472931
min       1.000000     1.000000
25%     699.000000   259.000000
50%    1397.000000   505.000000
75%    2095.000000   753.000000
max    2793.000000  1000.000000
___________________________

Наличие явных дубликатов в reviews
0
___________________________

Наличие пропусков в таблице reviews
review_id    0
book_id      0
username     0
text         0

In [12]:
preprocessing(publishers, 'publishers')

Посмотрим первые строки датафрейма publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


___________________________

Информация о таблице publishers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None
___________________________

Распределение данных в таблице publishers
       publisher_id
count    340.000000
mean     170.500000
std       98.293777
min        1.000000
25%       85.750000
50%      170.500000
75%      255.250000
max      340.000000
___________________________

Наличие явных дубликатов в publishers
0
___________________________

Наличие пропусков в таблице publishers
publisher_id    0
publisher       0
dtype: int64


**Во всех остальных таблицах всё корретно.**

<div style="border:solid green 2px; padding: 20px">
    
        Все данные корретно выгружены, можно приступать к исследованию данных.
    
</div>

___
## Анализ таблиц в БД. Ответы на поставленные вопросы
В ходе данного пункта будут даны ответы на 5 поставленных вопросов.

### Сколько книг вышло после 1 января 2000 года?

In [13]:
book_count = '''
    SELECT COUNT(book_id) AS books 
    FROM books 
    WHERE publication_date > CAST('2000-01-01' AS TIMESTAMP)
    '''
print('После 1 января 2000 года вышло книг:', int(query(book_count).values))

После 1 января 2000 года вышло книг: 819


### Количество обзоров и средняя оценка каждой книги

In [14]:
rev_rat = '''
    SELECT b.title AS title, COUNT(DISTINCT re.review_id) as review_count, AVG(ra.rating) as rating_avg
    FROM books as b
    FULL OUTER JOIN ratings as ra ON ra.book_id = b.book_id
    FULL OUTER JOIN reviews as re ON re.book_id = b.book_id
    GROUP BY b.book_id
    ORDER BY review_count DESC
    '''
display(query(rev_rat))

,title,review_count,rating_avg
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


In [15]:
rev_rat = '''
    SELECT b.title AS title, COUNT(DISTINCT re.review_id) as review_count, AVG(ra.rating) as rating_avg
    FROM books as b
    FULL OUTER JOIN ratings as ra ON ra.book_id = b.book_id
    FULL OUTER JOIN reviews as re ON re.book_id = b.book_id
    GROUP BY b.book_id
    ORDER BY rating_avg DESC
    '''
display(query(rev_rat))

,title,review_count,rating_avg
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


**Больше всего обзоров на книгу 'Twilight (Twilight #1)'. Есть много книг с рейтингом 5, однако не по всем из них много ревью.**

### Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [16]:
publisher = '''
    SELECT p.publisher, COUNT(b.book_id)
    FROM publishers AS p
    JOIN books AS b ON p.publisher_id=b.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher_id
    ORDER BY count(b.book_id) DESC
    LIMIT 1
    '''
print('Наибольшее число книг, а именно', query(publisher).values[0][1], 'книги, выпустило издательство', query(publisher).values[0][0])

Наибольшее число книг, а именно 42 книги, выпустило издательство Penguin Books


### Автор с самой высокой средней оценкой книг

In [17]:
author = '''
    SELECT a.author author, AVG(r.rating) rating
    FROM authors AS a
    JOIN books AS b ON a.author_id=b.author_id
    JOIN ratings AS r ON b.book_id=r.book_id
    
    WHERE b.book_id in
    (SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING count(rating) >= 50)
    
    GROUP BY author
    ORDER BY rating DESC
    LIMIT 1
    '''

print('Автором с самым высоким рейтингом, а именно', query(author).values[0][1], ', является', query(author).values[0][0])

Автором с самым высоким рейтингом, а именно 4.287096774193548 , является J.K. Rowling/Mary GrandPré


### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [18]:
user = '''
    SELECT AVG(re_count)
    FROM
    (SELECT username, COUNT(review_id) as re_count
    FROM reviews
    
    WHERE username in
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING count(rating_id) > 50)
    
    GROUP BY username) as cnt
    '''
query(user)
print('Среднее количество обзоров от активных пользователей:', float(query(user).values))

Среднее количество обзоров от активных пользователей: 24.333333333333332


___
## Общий вывод


<div style="border:solid green 2px; padding: 20px">
    
        После 1 января вышло 819 книг. Больше всего обзоров на книгу 'Twilight (Twilight #1)', при этом можно заметить, что выских рейтингов с большим количеством отзывов почти нет. Есть много книг с рейтингом 5, однако не по всем из них много ревью. Больше всего книг от издательства Penguin Books. Автором с самым высоким рейтингом является J.K. Rowling/Mary GrandPré. В среднем активные пользователи оставляют 24 обзора.
</div>